<a href="https://colab.research.google.com/github/parus-cristatus/tolokapizza/blob/main/tolokakit/async_write_aggregation_results_tsv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install toloka-kit

In [5]:
import asyncio
import time
import nest_asyncio
import csv
import toloka.async_client as async_toloka

In [6]:
target = 'SANDBOX'
# target = 'PRODUCTION'
token = 'XXXXXXXXXXXXXXXXXXXXXXXXXXXX'

async_toloka_client = async_toloka.AsyncTolokaClient(token=token, environment=target)

In [ ]:
nest_asyncio.apply()

start = time.time()

def write_tsv(tup: tuple):
    with open('aggr_results.tsv', 'w', newline='') as csv_file:
        fieldnames = ['input_values', 'confidence', 'output_values']
        writer = csv.DictWriter(csv_file, fieldnames=fieldnames, delimiter='\t')

        writer.writeheader()
        for item in zip(*tup):
            writer.writerow({'input_values': item[0].input_values, 'confidence': item[1][0], 'output_values': item[1][1]})

async def get_aggr_result():
    tasks = []
    solutions = []
    aggr_solutions = await async_toloka_client.get_aggregated_solutions('ba3eacb8-5vb9-4226-67ff-78h56d7a9220') #operation_id
    
    for solution in aggr_solutions:
        tasks.append(asyncio.create_task(async_toloka_client.get_task(task_id=solution.task_id)))
        solutions.append((solution.confidence, solution.output_values))
    input_values = await asyncio.gather(*tasks)
    
    return input_values, solutions

result = asyncio.run(get_aggr_result())
write_tsv(result)

print(f"{time.time() - start}")